In [1]:
import pandas as pd 
import numpy as np 

import torch
from torch import optim, argmax
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

C:\Users\leish\anaconda3\envs\tf3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MAX_RUNNERS = 16

In [1]:
X = 2

In [44]:
df = pd.read_csv("data/raw/full_data_with_features.csv")

df_runners_per_race = df.groupby(["race_id"]).size().reset_index(name="runners_per_race")
df = pd.merge(df, df_runners_per_race, on='race_id', how='left')

df.loc[(df['length'] == 0) & (df['won'] != 1), 'places'] = df['runners_per_race'].max()
df.loc[(df['length'] == 0) & (df['won'] != 1), 'length'] = df['length'].max()

In [45]:
df = df.drop([x for x in df.columns if "_og_" in x or "dam_sire" in x], axis=1)
df = df.drop(["sire_id", "dam_id", "date", "track_id","won","placed","places","pedigree_info","going_cat","trainer_ids","jockey_ids"], axis=1)

In [46]:
race_ids = df['race_id'].unique().repeat(16)
last_horse_id = df['horse_ids'].max()
df = df.append(pd.DataFrame({'race_id':race_ids, 
                             'horse_ids':np.array([last_horse_id + x for x in range(len(race_ids))]),
                             'length': df['length'].max() + 10}))

C:\Users\leish\AppData\Local\Temp\ipykernel_24192\2429516955.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'race_id':race_ids,


In [47]:
df

C:\Users\leish\anaconda3\envs\tf3\lib\site-packages\IPython\lib\pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
C:\Users\leish\anaconda3\envs\tf3\lib\site-packages\IPython\core\formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,race_id,horse_ids,race_class,distance,going,race_age,race_handicap,race_type,draws,horse_ages,...,length,sire_win_percent,dam_win_percent,sire_prog_going_win_percent,sire_prog_type_win_percent,sire_prog_dist_win_percent,dam_prog_going_win_percent,dam_prog_type_win_percent,dam_prog_dist_win_percent,runners_per_race
0,495885.0,716693,5.0,2420.0,18.0,32.0,False,1.0,5.0,9.0,...,0.00,0.081235,0.122302,0.081140,0.055738,0.070175,0.081140,0.055738,0.070175,7.0
1,495885.0,708440,5.0,2420.0,18.0,32.0,False,1.0,3.0,9.0,...,1.75,0.096535,0.151515,0.066667,0.100775,0.092784,0.066667,0.100775,0.092784,7.0
2,495885.0,718346,5.0,2420.0,18.0,32.0,False,1.0,7.0,9.0,...,4.75,0.111782,0.038462,0.101523,0.102041,0.142857,0.101523,0.102041,0.142857,7.0
3,495885.0,744643,5.0,2420.0,18.0,32.0,False,1.0,6.0,9.0,...,5.75,0.059289,0.000000,0.025641,0.047337,0.000000,0.025641,0.047337,0.000000,7.0
4,495885.0,714689,5.0,2420.0,18.0,32.0,False,1.0,4.0,10.0,...,7.25,0.060606,0.013889,0.044444,0.072193,0.079646,0.044444,0.072193,0.079646,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3021643,783410.0,6555143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,337.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3021644,783410.0,6555144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,337.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3021645,783410.0,6555145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,337.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3021646,783410.0,6555146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,337.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
max_df = df.groupby(['race_id']).max().reset_index()[['race_id','race_class', 'distance', 'going','race_age','race_handicap','race_type','draws','horse_ages']]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [49]:
df = df.drop(['race_class', 'distance', 'going','race_age','race_handicap','race_type','draws','horse_ages'], axis=1)

In [50]:
df = df.merge(max_df, on="race_id")

In [51]:
df = df.sort_values(by='length',ascending=True).groupby('race_id').head(16)

In [52]:
df = df.groupby(['race_id','horse_ids']).mean().reset_index()

In [53]:
df['id_in_race'] = np.tile(np.arange(16), len(df['race_id'].unique())).astype("str")

In [55]:
df

,race_id,horse_ids,horse_weight,horse_win_percents,jockey_win_percent,trainer_win_percent,days_since_last_race,mean_speed_figures,last_figures,best_figures_dist,...,runners_per_race,race_class,distance,going,race_age,race_handicap,race_type,draws,horse_ages,id_in_race
0,253375.0,708201,117.0,0.166667,0.10,0.15,22.0,48.757096,18.000000,79.000000,...,13.0,-1.0,1540.0,20.0,30.0,1.0,1.0,13.0,12.0,0
1,253375.0,735766,129.0,0.166667,0.05,0.10,8.0,51.800000,50.000000,97.059879,...,13.0,-1.0,1540.0,20.0,30.0,1.0,1.0,13.0,12.0,1
2,253375.0,738107,133.0,0.333333,0.00,0.00,22.0,50.600000,23.000000,76.000000,...,13.0,-1.0,1540.0,20.0,30.0,1.0,1.0,13.0,12.0,2
3,253375.0,756693,121.0,0.000000,0.10,0.15,157.0,51.280551,43.000000,82.000000,...,13.0,-1.0,1540.0,20.0,30.0,1.0,1.0,13.0,12.0,3
4,253375.0,762614,127.0,0.000000,0.10,0.05,29.0,41.778762,16.000000,66.000000,...,13.0,-1.0,1540.0,20.0,30.0,1.0,1.0,13.0,12.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3021643,783410.0,2219533,155.0,0.000000,0.05,0.05,424.0,90.535997,78.287269,123.774746,...,15.0,-1.0,4070.0,19.0,41.0,1.0,0.0,0.0,15.0,11
3021644,783410.0,3093289,151.0,0.000000,0.20,0.00,196.0,52.501068,0.000000,105.002136,...,15.0,-1.0,4070.0,19.0,41.0,1.0,0.0,0.0,15.0,12
3021645,783410.0,3533498,155.0,NaN,0.15,0.00,NaN,NaN,NaN,NaN,...,15.0,-1.0,4070.0,19.0,41.0,1.0,0.0,0.0,15.0,13
3021646,783410.0,3533500,155.0,NaN,0.10,0.20,NaN,NaN,NaN,NaN,...,15.0,-1.0,4070.0,19.0,41.0,1.0,0.0,0.0,15.0,14


In [56]:
df = df.drop(['horse_ids'], axis=1)

In [57]:
df = df.pivot(index=['race_id','going','distance','race_age','race_handicap','race_type'], columns='id_in_race')

In [58]:
df.columns = df.columns.map("_".join)
df = df.reset_index()

In [59]:
df = df.fillna(0)

In [60]:
df = df.drop(['race_id'], axis=1)

In [61]:
X = df.loc[:, ~df.columns.str.contains("length")]
y = df.loc[:, df.columns.str.contains("length")]

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3)

In [63]:
ss = StandardScaler()
train_data = ss.fit_transform(X_train)
test_data = ss.transform(X_test)

In [20]:
class HorseDataset(Dataset):
    def __init__(self, X, y, batch_size=64, shuffle=True):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X_val = self.X.iloc[idx]
        y_val = self.y.iloc[idx]
        
        X_val = torch.from_numpy(np.array([X_val])).to(torch.float32)
        y_val = torch.from_numpy(np.array([y_val])).to(torch.float32)
        
        return {"X":X_val, "y":y_val}

In [21]:
train_loader = HorseDataset(X_train,y_train, batch_size=64, shuffle=True)
test_loader = HorseDataset(X_test, y_test, batch_size=64, shuffle=True)

In [25]:
class BasicNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(in_features=613, out_features=100)
        self.hidden_1 = nn.Linear(in_features=100, out_features=20)
        self.output = nn.Linear(in_features=20, out_features=16)
    
    def forward(self, x):
        x = nn.functional.relu(self.input(x))
        x = nn.functional.relu(self.hidden_1(x))
        return self.output(x)

In [ ]:
num_epochs = 10
train_accuracies, test_accuracies = [], []

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    # Train set
    for i in range(len(train_loader)):
        X = train_loader[i]['X']
        y = train_loader[i]['y']
        preds = model(X)
        pred_labels = torch.argmax(preds, axis=1)
        loss = loss_function(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_accuracies.append(
        100 * torch.mean((pred_labels == y).float()).item()
    )
    
    # Test set
    for i in range(len(test_loader)):
        X = test_loader[i]['X']
        y = test_loader[i]['y']
        pred_labels = torch.argmax(model(X), axis=1)
        test_accuracies.append(
            100 * torch.mean((pred_labels == y).float()).item()
        )

In [ ]:
model = BasicNN()
model

In [77]:
X_train.shape

(132197, 597)

In [ ]:
import tensorflow as tf

In [78]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(600, activation='relu'),
      tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(600, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(16)
])


In [79]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])


In [80]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
4132/4132 [==============================] - 26s 6ms/step - loss: 4367.3877 - accuracy: 0.2005
Epoch 2/10
4132/4132 [==============================] - 27s 6ms/step - loss: 3148.1152 - accuracy: 0.2244
Epoch 3/10
4132/4132 [==============================] - 26s 6ms/step - loss: 2954.9653 - accuracy: 0.2142
Epoch 4/10
4132/4132 [==============================] - 26s 6ms/step - loss: 2863.0417 - accuracy: 0.2081
Epoch 5/10
4132/4132 [==============================] - 26s 6ms/step - loss: 2801.4949 - accuracy: 0.2051
Epoch 6/10
4132/4132 [==============================] - 29s 7ms/step - loss: 2762.2786 - accuracy: 0.2087
Epoch 7/10
4132/4132 [==============================] - 27s 7ms/step - loss: 2721.7583 - accuracy: 0.2031
Epoch 8/10
4132/4132 [==============================] - 27s 7ms/step - loss: 2697.2600 - accuracy: 0.2048
Epoch 9/10
4132/4132 [==============================] - 28s 7ms/step - loss: 2679.0693 - accuracy: 0.2033
Epoch 10/10
4132/4132 [=======================

In [71]:
X_train.columns[X_train.dtypes != np.float64]

Index([], dtype='object')